In [6]:
!pip install silence_remover

  Using cached silence_remover-1.0.0-py3-none-any.whl (7.9 kB)


In [1]:
import pickle
import shutil
import csv
import os
import librosa
import librosa.display
import pyaudio
import wave
import keras
import pyaudio  
import sys
import numpy as np
import matplotlib.pyplot as plt
import speech_recognition as speech_r
import scipy.signal as signal
import random as rn
import tensorflow as tf
import pandas as pd
from pydub import AudioSegment
from time import sleep
from matplotlib import cm
from python_speech_features import logfbank, fbank
from scipy.fftpack import dct
from silence_remover.silence_detector import SilenceDetector
from silence_remover.filter_generator import FilterGenerator
from silence_remover.media_editor import MediaEditor
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
from sklearn.linear_model import LogisticRegression
from keras.layers import Dense
from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from keras.layers import Dense,concatenate, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, MaxPooling1D,Multiply,Conv1D, Conv2D, MaxPooling2D
from playsound import playsound
from sklearn.model_selection import train_test_split

C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\thakd\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\thakd\anaconda3\envs\UI\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [11]:
DATA_PATH = "./train_mfcc/"
X_train = [] 
X_test = []
Y_train = []
Y_test = []
tf_classes = 0
labels = []
X_data = []
Y_label = []

    
def load_npy(save_path) : 
    
    # 데이터들을 저장할 배열
    
    global X_train, X_test, Y_train, Y_test, tf_classes
    
    folders = os.listdir(save_path) ## 경로에 있는 폭더 목록
    
    for folder in folders :
        
        if not os.path.isdir(save_path) : continue 
            
        files = os.listdir(save_path + "/" + folder)        
        print("Foldername : ", folder, "-", len(files),"파일")
        
        for file in files:
            if not file.endswith(".npy") : continue ## 미리 전처리하여 변환한 npy불러오기
            
            else:               
                mfcc = np.load(save_path + "/" + folder + "/" + file) ## mfcc변환한 파일 값 저장
                X_data.extend(mfcc)
                
                label = [0 for i in range(len(folders))]
                label[tf_classes] = 1 ## 폴더 순서대로 라벨링
        
                for i in range(len(mfcc)) :
                    Y_label.append(label)
                    
        tf_classes = tf_classes + 1 ## 클래스번호 부여

    print("X_data : ", np.shape(X_data))
    print("Y_label : ", np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label), test_size = 0.2, stratify = np.array(Y_label))
    xy = (X_train, X_test, Y_train, Y_test)

In [12]:
load_npy(DATA_PATH)

print(tf_classes,"개의 클래스!!")
print("X_train : ", np.shape(X_train))
print("Y_train : ", np.shape(Y_train))
print("X_test : ", np.shape(X_test))
print("Y_test : ", np.shape(Y_test))

X_train = np.expand_dims(X_train, -1) # 차수 늘리기 >> Conv1D는 3개의 텐서가 필요 
X_test = np.expand_dims(X_test, -1)
print(X_train.shape)
print(X_test.shape)

Foldername :  강은기 - 19 파일
Foldername :  강은서 - 19 파일
Foldername :  김나리 - 19 파일
Foldername :  김엄지 - 19 파일
Foldername :  김용준 - 19 파일
Foldername :  윤동준 - 19 파일
Foldername :  이성민 - 19 파일
Foldername :  이재빈 - 19 파일
Foldername :  정승화 - 19 파일
Foldername :  주다빈 - 19 파일
Foldername :  주홍식 - 19 파일
X_data :  (46336, 19)
Y_label :  (46336, 11)
11 개의 클래스!!
X_train :  (37068, 19)
Y_train :  (37068, 11)
X_test :  (9268, 19)
Y_test :  (9268, 11)
(37068, 19, 1)
(9268, 19, 1)


In [4]:
early_stopping = EarlyStopping(patience = 30)

In [6]:
datalist1 = ["./test_mfcc/강은기_19.npy", "./test_mfcc/강은서_19.npy", "./test_mfcc/김나리_19.npy",
             "./test_mfcc/김엄지_19.npy", "./test_mfcc/김용준_19.npy", "./test_mfcc/윤동준_19.npy",
            "./test_mfcc/이성민_19.npy", "./test_mfcc/이재빈_19.npy", "./test_mfcc/정승화_19.npy",
            "./test_mfcc/주다빈_19.npy", "./test_mfcc/주홍식_19.npy"]

In [5]:
input_layer = tf.keras.Input(X_train[0].shape)
conv_top = tf.keras.layers.Conv1D(128,
                                   kernel_size = 4,
                                   strides = 3,
                                   padding = 'same',
                                   kernel_initializer = 'glorot_uniform',
                                   kernel_regularizer = tf.keras.regularizers.L2(l=0.0001))(input_layer)
conv_top = BatchNormalization()(conv_top)
conv_top = tf.keras.layers.Activation('relu')(conv_top)
conv_top = tf.keras.layers.MaxPool1D(pool_size = 2,
                                     strides = None,
                                     padding = 'same')(conv_top)
    
conv_top = tf.keras.layers.Conv1D(128, kernel_size = 4, 
                                   strides = 1,
                                   padding = 'same',
                                   kernel_initializer = 'glorot_uniform',
                                   kernel_regularizer = tf.keras.regularizers.L2(l=0.0001))(conv_top)
conv_top = BatchNormalization()(conv_top)
conv_top = tf.keras.layers.Activation('relu')(conv_top)
conv_top = tf.keras.layers.MaxPool1D(pool_size = 2, 
                                     strides = None,
                                     padding = 'same')(conv_top)
    
conv_top = tf.keras.layers.Conv1D(256, kernel_size = 3,
                                   strides = 1,
                                   padding = 'same',
                                   kernel_initializer = 'glorot_uniform',
                                   kernel_regularizer = tf.keras.regularizers.L2(l=0.0001))(conv_top)
conv_top = BatchNormalization()(conv_top)
conv_top = tf.keras.layers.Activation('relu')(conv_top)
conv_top = tf.keras.layers.MaxPool1D(pool_size = 2, 
                                     strides = None,
                                     padding = 'same')(conv_top)
    
conv_top = tf.keras.layers.Conv1D(512, kernel_size = 2, 
                                   strides = 1,
                                   padding = 'same',
                                   kernel_initializer = 'glorot_uniform',
                                   kernel_regularizer = tf.keras.regularizers.L2(l=0.0001))(conv_top)
conv_top = BatchNormalization()(conv_top)
conv_top = tf.keras.layers.Activation('relu')(conv_top)
conv_top = tf.keras.layers.MaxPool1D(pool_size = 2, strides = None, padding = 'same')(conv_top)

conv_top = tf.keras.models.Model(input_layer, conv_top)
conv_top.summary()

conv_top.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',             
             metrics = ['accuracy'])





Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 7, 128)            640       
                                                                 
 batch_normalization (BatchN  (None, 7, 128)           512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 7, 128)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 128)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 128)            65664 

In [13]:
top_model = Sequential()
top_model.add(conv_top)
top_model.add(Flatten())
top_model.add(Dense(64, activation = 'relu'))
top_model.add(Dense(tf_classes, activation = 'softmax'))
top_model.summary()

top_model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',             
             metrics = ['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 1, 512)            431616    
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 11)                715       
                                                                 
Total params: 465,163
Trainable params: 33,547
Non-trainable params: 431,616
_________________________________________________________________


In [8]:
# 모델 구현 시간 체크
import time
# 시작시간 체크
start = time.time()
history = top_model.fit(X_train, Y_train, epochs = 100, batch_size = 100, validation_data = (X_test, Y_test), callbacks=[early_stopping])

Epoch 1/100
304/304 [==============================] - 6s 16ms/step - loss: 0.6612 - accuracy: 0.7865 - val_loss: 0.7966 - val_accuracy: 0.7562
Epoch 2/100
304/304 [==============================] - 5s 15ms/step - loss: 0.4973 - accuracy: 0.8435 - val_loss: 0.5474 - val_accuracy: 0.8269
Epoch 3/100
304/304 [==============================] - 5s 15ms/step - loss: 0.4415 - accuracy: 0.8617 - val_loss: 0.4974 - val_accuracy: 0.8445
Epoch 4/100
304/304 [==============================] - 5s 16ms/step - loss: 0.4055 - accuracy: 0.8761 - val_loss: 0.5803 - val_accuracy: 0.8221
Epoch 5/100
304/304 [==============================] - 5s 17ms/step - loss: 0.3751 - accuracy: 0.8879 - val_loss: 0.4602 - val_accuracy: 0.8653
Epoch 6/100
304/304 [==============================] - 5s 15ms/step - loss: 0.3546 - accuracy: 0.8965 - val_loss: 0.4158 - val_accuracy: 0.8810
Epoch 7/100
304/304 [==============================] - 5s 16ms/step - loss: 0.3385 - accuracy: 0.8992 - val_loss: 0.4368 - val_accuracy:

In [ ]:
# 모델 종료 시간 확인
end = time.time()
# 모델 구동 시간 계산
print(f"모델구동시간 : {end-start} sec.")

In [9]:
## 사용자 화자인식 테스트

num = 0
for i in datalist1 :

    XX_test = np.load(i)
    YY_test = []

    label = [0 for i in range(tf_classes)] # class의 개수만큼 y_test만드는 과정
    label[num] = 1

    for i in range(len(XX_test)) :
        YY_test.append(label)

    XX_train = np.array(X_train)
    XX_test = np.array(XX_test)
    YY_train = np.array(Y_train)
    YY_test = np.array(YY_test)
    print(num)
    XX_test = np.expand_dims(XX_test, -1)

    top_model.evaluate(XX_test, YY_test)
    num += 1

0
8/8 [==============================] - 0s 3ms/step - loss: 0.5883 - accuracy: 0.8884
1
6/6 [==============================] - 0s 3ms/step - loss: 0.4973 - accuracy: 0.8830
2
6/6 [==============================] - 0s 5ms/step - loss: 1.2867 - accuracy: 0.7604
3
8/8 [==============================] - 0s 2ms/step - loss: 0.1071 - accuracy: 0.9957
4
8/8 [==============================] - 0s 4ms/step - loss: 0.4039 - accuracy: 0.9246
5
5/5 [==============================] - 0s 4ms/step - loss: 0.5159 - accuracy: 0.8473
6
6/6 [==============================] - 0s 4ms/step - loss: 2.2037 - accuracy: 0.5636
7
6/6 [==============================] - 0s 4ms/step - loss: 0.4493 - accuracy: 0.9412
8
6/6 [==============================] - 0s 3ms/step - loss: 0.8029 - accuracy: 0.8448


IndexError: list assignment index out of range

In [10]:
conv_top.trainable = False

In [14]:
# 모델 구현 시간 체크
import time
# 시작시간 체크
start = time.time()
history = top_model.fit(X_train, Y_train, epochs = 100, batch_size = 100, validation_data = (X_test, Y_test), callbacks=[early_stopping])

Epoch 1/100
371/371 [==============================] - 3s 7ms/step - loss: 0.6125 - accuracy: 0.8333 - val_loss: 0.4466 - val_accuracy: 0.8801
Epoch 2/100
371/371 [==============================] - 2s 6ms/step - loss: 0.4168 - accuracy: 0.8943 - val_loss: 0.4107 - val_accuracy: 0.8913
Epoch 3/100
371/371 [==============================] - 2s 6ms/step - loss: 0.3836 - accuracy: 0.9038 - val_loss: 0.3947 - val_accuracy: 0.8976
Epoch 4/100
371/371 [==============================] - 2s 6ms/step - loss: 0.3602 - accuracy: 0.9105 - val_loss: 0.3846 - val_accuracy: 0.9005
Epoch 5/100
371/371 [==============================] - 2s 5ms/step - loss: 0.3456 - accuracy: 0.9163 - val_loss: 0.3851 - val_accuracy: 0.9015
Epoch 6/100
371/371 [==============================] - 2s 6ms/step - loss: 0.3325 - accuracy: 0.9205 - val_loss: 0.3754 - val_accuracy: 0.9031
Epoch 7/100
371/371 [==============================] - 2s 6ms/step - loss: 0.3195 - accuracy: 0.9247 - val_loss: 0.3735 - val_accuracy: 0.9058

In [15]:
## 사용자 화자인식 테스트

num = 0
for i in datalist1 :

    XX_test = np.load(i)
    YY_test = []

    label = [0 for i in range(tf_classes)] # class의 개수만큼 y_test만드는 과정
    label[num] = 1

    for i in range(len(XX_test)) :
        YY_test.append(label)

    XX_train = np.array(X_train)
    XX_test = np.array(XX_test)
    YY_train = np.array(Y_train)
    YY_test = np.array(YY_test)
    print(num)
    XX_test = np.expand_dims(XX_test, -1)

    top_model.evaluate(XX_test, YY_test)
    num += 1

0
8/8 [==============================] - 0s 4ms/step - loss: 0.9053 - accuracy: 0.8595
1
6/6 [==============================] - 0s 3ms/step - loss: 0.8411 - accuracy: 0.8538
2
6/6 [==============================] - 0s 3ms/step - loss: 1.6538 - accuracy: 0.7552
3
8/8 [==============================] - 0s 3ms/step - loss: 0.1032 - accuracy: 1.0000
4
8/8 [==============================] - 0s 3ms/step - loss: 0.4465 - accuracy: 0.9365
5
5/5 [==============================] - 0s 3ms/step - loss: 0.8431 - accuracy: 0.8397
6
6/6 [==============================] - 0s 3ms/step - loss: 2.3311 - accuracy: 0.6424
7
6/6 [==============================] - 0s 3ms/step - loss: 0.9966 - accuracy: 0.8706
8
6/6 [==============================] - 0s 3ms/step - loss: 0.8964 - accuracy: 0.8333
9
3/3 [==============================] - 0s 3ms/step - loss: 0.8423 - accuracy: 0.8358
10
7/7 [==============================] - 0s 3ms/step - loss: 0.5979 - accuracy: 0.8539


In [150]:
model = load_model("./모델/model.h5")

In [151]:
model.save("./모델/model.h5")